### Importation des bibliothèques nécessaires 

In [43]:
import requests
import pandas as pd
from tqdm import tqdm
import deepl
import os

In [44]:
#Dossier de destination
outputFolder="outputFolder"

# Collecte des personnages de Dragon Ball à laide de la DragonBallAPI

In [ ]:
#Endpoint dragon ball API
baseUrl="https://dragonball-api.com/api"

#nombre total de personnage
maxId=80

#Paramètre des charactères de l'API dragon ball
characterParameter="/characters/"

#Liste vide de personnages
charactersList=[]

#Boucle pour qui fait varier la valeur de i de 1 à 80
for i in tqdm(range(1,maxId),desc="Chargement des personnages de Dragon Ball:-)"):
    try:
        #Requête Get
        characterRequest=requests.get(f"https://dragonball-api.com/api{characterParameter}{i}")
        if characterRequest.status_code==200:
            characterRequest=characterRequest.json()
            character= dict(list(characterRequest.items())[:8])
            character["idPlanet"]=characterRequest["originPlanet"]["id"]
            charactersList.append(character)
    except OSError as error:
            print("Désolé une erreur s'est produite lors du chargement de ce personnage :-(")
            print(f"Type d'erreur : {TypeError} ")
            print(f"Nom de l'erreur : {NameError} ")
            print(f"index de l'erreur : {IndexError} ")
            print(error)

In [ ]:
charactersDataFrame=pd.DataFrame(charactersList)

#Nettoyage de la colonne Ki et maxKi
charactersDataFrame["ki"]=charactersDataFrame["ki"].replace([r"\.", r"\s", r",",r"(?i)Billion",r"(?i)Septillion","unknown",r"(?i)quadrillion",r"(?i)Quintillion",r"(?i)Googolplex",r"(?i)Trillion"],
                                                            ["","","","0000000","000000000000","","000000000","0000000000","00000000000000","00000000"],regex=True)

charactersDataFrame["maxKi"]=charactersDataFrame["maxKi"].replace([r"\.", r"\s", r",",r"(?i)Billion",r"(?i)Septillion","unknown",r"(?i)quadrillion",r"(?i)Quintillion",r"(?i)Googolplex",r"(?i)Trillion",r"(?i)Sextillion",r"(?i)Septllion"],
                                                            ["","","","0000000","000000000000","","000000000","0000000000","00000000000000","00000000","00000000000","000000000000"],regex=True)
auth_key = os.getenv("deeplKey")
translator = deepl.Translator(auth_key)
#Traduction de la colonne description à l'aide de l'API de Deepl
charactersDataFrame["description"]=charactersDataFrame["description"].apply(lambda x: translator.translate_text(x,target_lang="FR"))
charactersDataFrame
charactersDataFrame.to_csv(f"{outputFolder}/DragonBallCharacter.csv",";",index=False)

# Collecte des transformations de Dragon Ball à laide de la DragonBallAPI

In [ ]:
#Liste vide des transformations des personnages
transformationsList=[]

for i in tqdm(range(1,maxId),desc="Chargement des transformations des personnages de Dragon Ball:-)"):
    try:
        #Requête Get
        characterRequest=requests.get(f"https://dragonball-api.com/api{characterParameter}{i}")
        if characterRequest.status_code==200:
            characterRequest=characterRequest.json()
            transformationscharacter= characterRequest["transformations"]
            if transformationscharacter:
                for transformation in transformationscharacter:
                    transformation["idCharacter"]=characterRequest["id"]
                    transformationsList.append(transformation)
    except OSError as error:
            print(f"Désolé une erreur s'est produite lors du chargement des transformations du personnage {i} :-(")
            print(f"Type d'erreur : {TypeError} ")
            print(f"Nom de l'erreur : {NameError} ")
            print(f"index de l'erreur : {IndexError} ")
            print(error)

In [49]:
transformationsDataFrames=pd.DataFrame(transformationsList)
del transformationsDataFrames["deletedAt"]
transformationsDataFrames["ki"]=transformationsDataFrames["ki"].replace([r"\.", r"\s", r",",r"(?i)Billion",r"(?i)Septillion","unknown",r"(?i)quadrillion",r"(?i)Quintillion",r"(?i)Googolplex",r"(?i)Trillion",r"(?i)Sextillion",r"(?i)Septllion",r"(?i)octillion"],
                                                            ["","","","0000000","000000000000","","000000000","0000000000","00000000000000","00000000","00000000000","000000000000","0000000000000"],regex=True)

transformationsDataFrames
transformationsDataFrames.to_csv(f"{outputFolder}/DragonBallTransformations.csv",";",index=False)

# Collecte des Planetes de Dragon Ball à laide de la DragonBallAPI

In [50]:
#Paramètre des charactères de l'API dragon ball
planetParameter="/planets"

#Requête Get
planetRequest=requests.get(f"https://dragonball-api.com/api{planetParameter}")
planetRequest=planetRequest.json()
planetDataFrame=pd.json_normalize(planetRequest["items"])
del planetDataFrame["deletedAt"]

In [51]:
planetDataFrame["description"]=planetDataFrame["description"].apply(lambda x: translator.translate_text(x,target_lang="FR"))
planetDataFrame
planetDataFrame.to_csv(f"{outputFolder}/DragonBallPlanet.csv",";",index=False)